## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,75,13.80,broken clouds
1,1,Puertollano,ES,38.6871,-4.1073,85.96,17,0,5.26,clear sky
2,2,Hudiksvall,SE,61.7290,17.1036,60.53,82,67,6.89,broken clouds
3,3,Reyes,BO,-14.3167,-67.3833,94.78,26,3,5.03,clear sky
4,4,Tari,NG,12.4086,4.4914,82.17,69,100,9.64,overcast clouds


### Ask for customer input and convert the input to a floating number.

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (in F) would you like for your trip?  "))
max_temp = float(input("What is the maximum temperature (in F) would you like for your trip?  "))

What is the minimum temperature (in F) would you like for your trip?  75
What is the maximum temperature (in F) would you like for your trip?  90


### Create a new DataFrame using loc method. Determine and remove empty rows.

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
vacation_city_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puertollano,ES,38.6871,-4.1073,85.96,17,0,5.26,clear sky
4,4,Tari,NG,12.4086,4.4914,82.17,69,100,9.64,overcast clouds
6,6,Guymon,US,36.6828,-101.4816,89.47,40,1,5.46,clear sky
7,7,Albany,US,42.6001,-73.9662,88.03,56,17,3.71,few clouds
11,11,Port Blair,IN,11.6667,92.7500,82.18,83,100,11.16,overcast clouds
25,25,Faanui,PF,-16.4833,-151.7500,78.91,74,9,13.69,clear sky
26,26,Savannah Bight,HN,16.4500,-85.8500,89.65,79,100,19.71,overcast clouds
29,29,Komsomolskiy,UZ,40.4272,71.7189,80.02,20,0,5.08,clear sky
30,30,Porto Novo,BJ,6.4965,2.6036,76.12,87,100,10.29,overcast clouds
33,33,Hilo,US,19.7297,-155.0900,78.53,74,90,8.05,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.

# Doesn't seem to work. Try isnull().sum()
vacation_city_df.count()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
# Determine the null values using isnull().sum()

vacation_city_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


#### There is no null rows here. 
#### We would use dropna() method to remove the empty rows in in Step #3
#### The command would be saved in a different DataFrame: clean_vacation_df = vacation_city_df.dropna( )

### Create a Hotel Information DataFrame with relevant information

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puertollano,ES,85.96,clear sky,38.6871,-4.1073,
4,Tari,NG,82.17,overcast clouds,12.4086,4.4914,
6,Guymon,US,89.47,clear sky,36.6828,-101.4816,
7,Albany,US,88.03,few clouds,42.6001,-73.9662,
11,Port Blair,IN,82.18,overcast clouds,11.6667,92.7500,
25,Faanui,PF,78.91,clear sky,-16.4833,-151.7500,
26,Savannah Bight,HN,89.65,overcast clouds,16.4500,-85.8500,
29,Komsomolskiy,UZ,80.02,clear sky,40.4272,71.7189,
30,Porto Novo,BJ,76.12,overcast clouds,6.4965,2.6036,
33,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add location parameter --> Latidue and Longitude to a Location key --FMJ
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping")        
        

Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping


###### There are cities with no hotels. Need to clean it up. I will use the loc[ ] method.

In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puertollano,ES,85.96,clear sky,38.6871,-4.1073,Hotel Puertollano
4,Tari,NG,82.17,overcast clouds,12.4086,4.4914,
6,Guymon,US,89.47,clear sky,36.6828,-101.4816,Super 8 Guymon
7,Albany,US,88.03,few clouds,42.6001,-73.9662,
11,Port Blair,IN,82.18,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


#### These are the cities with no hotels

In [11]:
# These are the cities with no hotel
no_hotel = hotel_df.loc[(hotel_df["Hotel Name"] == "")]

no_hotel.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tari,NG,82.17,overcast clouds,12.4086,4.4914,
7,Albany,US,88.03,few clouds,42.6001,-73.9662,
46,Bukama,CD,76.51,clear sky,-9.2000,25.8500,
107,Lodja,CD,77.07,overcast clouds,-3.4833,23.4333,
180,Bonfim,BR,84.16,overcast clouds,3.0833,-59.9500,


### A new Clean DataFrame containing cities with hotels. 

In [12]:
# 7. Drop the rows where there is no Hotel Name.
# Make a clean hotel data frame.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puertollano,ES,85.96,clear sky,38.6871,-4.1073,Hotel Puertollano
6,Guymon,US,89.47,clear sky,36.6828,-101.4816,Super 8 Guymon
11,Port Blair,IN,82.18,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
25,Faanui,PF,78.91,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
26,Savannah Bight,HN,89.65,overcast clouds,16.4500,-85.8500,Villa on Dunbar Rock
...,...,...,...,...,...,...,...
672,Tuy Hoa,VN,82.47,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
673,Mailani,IN,80.42,light rain,28.2833,80.3500,Nani House
674,Sur,OM,88.86,clear sky,22.5667,59.5289,Sur Plaza Hotel
675,Kipini,KE,75.49,clear sky,-2.5257,40.5262,Delta Sea Mall


### Generate an Output File (CSV)

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \N{DEGREE SIGN} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

### Plot the Data and save the figure

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# 11b. Display the figure
# Assign the figure variable.
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))